In [1]:
#Libraries to import for function calls
import csv
import os
import cv2
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

#Code to read in the driving_log.csv file from the simulation
#Will probably need to read in mutiple driving logs, as I have 4 different driving logs from different events
car_angles = []
steering_angles = []
samples = []
with open('./Simulation_Recordings/Course 1 Simulation/driving_log.csv' , 'r') as csvfile: #First CSV FIle path
    reader = csv.reader(csvfile)
    for line in reader:
            samples.append(line)

with open('./Simulation_Recordings/Course 2 Simulation/driving_log.csv', 'r') as csvfile: #Second CSV FIle path
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

with open('./Simulation_Recordings/Lane Correct Simulation/driving_log.csv' , 'r') as csvfile: #Third CSV FIle path
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

with open('./Simulation_Recordings/Smooth Curve Simulation/driving_log.csv' , 'r') as csvfile: #Fourth CSV FIle path
    reader = csv.reader(csvfile)
    for line in reader:
            samples.append(line)
            
with open('./Simulation_Recordings/data/driving_log.csv' , 'r') as csvfile: #Fifth CSV FIle path
    reader = csv.reader(csvfile)
    for line in reader:
            samples.append(line)

#split up training and validation samples
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2) #need to correct variable used "samples"

#Test image uploads
#new_filepath = '/home/carnd/Simulation_Recordings/Course 1 Simulation/IMG/center_2017_08_16_01_08_03_977.jpg'
#print(samples[0][0].split('\Simulation Recordings')[-1])
#name = samples[0][0].split('\Simulation Recordings')[-1]
#filepath = '/home/carnd/Simulation_Recordings' + name
#new_filepath = filepath.replace("\\", "/")
#print(new_filepath)
#image = cv2.imread(new_filepath)
#print(image.shape)
#break

def generator(samples, batch_size=32, filepath= '/home/carnd/Simulation_Recordings'):
    num_samples = len(samples)
    correction = 0.2 # this is a parameter to tune
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                #Collect Center Images & Center Image Steering Angle
                if '\Simulation Recordings' in batch_sample[0]:
                    name = filepath + batch_sample[0].split('\Simulation Recordings')[-1]
                else: 
                        name = filepath + '/data/' + batch_sample[0]
                new_name = name.replace("\\", "/")
                center_image = cv2.imread(new_name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                #Create and collect inverted center images & center image steering angles for more data
                images.append(cv2.flip(center_image,1))
                angles.append(center_angle*-1.0)
                #Collect left images & left image steering angles
                if '\Simulation Recordings' in batch_sample[1]:
                    name = filepath + batch_sample[1].split('\Simulation Recordings')[-1]
                else:
                    #Data in left and right images label has an unwanted space
                    name_no_space = batch_sample[1].split(' ')[-1]
                    name = filepath + '/data/' + name_no_space
                new_name = name.replace("\\", "/")
                left_image = cv2.imread(new_name)
                left_angle = float(batch_sample[3]) + correction
                images.append(left_image)
                angles.append(left_angle)
                #Create and collect inverted left images & left image steering angles for more data
                images.append(cv2.flip(left_image,1))
                angles.append(left_angle*-1.0)
                #Collect right images & right image steering angles
                if '\Simulation Recordings' in batch_sample[2]:
                    name = filepath + batch_sample[2].split('\Simulation Recordings')[-1]
                else:
                    #Data in left and right images label has an unwanted space
                    name_no_space = batch_sample[2].split(' ')[-1]
                    name = filepath + '/data/' + name_no_space
                new_name = name.replace("\\", "/")
                right_image = cv2.imread(new_name)
                right_angle = float(batch_sample[3]) - correction
                images.append(right_image)
                angles.append(right_angle)
                #Create and collect inverted right images & right image steering angles for more data
                images.append(cv2.flip(right_image,1))
                angles.append(center_angle*-1.0)
           # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train,y_train)



# compile and train the model using the generator function
batch_size = 192
train_generator = generator(train_samples, batch_size=batch_size) #do i need to change the batch size back to 32?
validation_generator = generator(validation_samples, batch_size=batch_size) #do i need to change the batch size back to 32?

#Import Keras libraries to utilize easy classifier functions
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D


#Create Regression Model
model = Sequential()
#Cropping Layer, crops down 70 pixels from top, crops 25 pixels from bottom, and zero pixels from either side
model.add(Cropping2D(cropping=((70,25), (0,0)),input_shape=(160,320,3)))
#Lambda Layer to process the image before hand, i.e. normalize the pixel values
#model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(3, 160, 320), output_shape=(3, 160, 320)))
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320,3))) #They did x/127.5 -1 in another example
#Regression steps
#Use dropout or pooling layers to reduce overfitting
#Use fewer convolutions or fewer fully connected layers to reduce overfitting
model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))


#Validate the model
model.compile(loss = 'mse', optimizer = 'adam')

#Plot the results
history_object = model.fit_generator(train_generator, samples_per_epoch = 6*len(train_samples), validation_data = validation_generator, nb_val_samples = len(validation_samples), nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

#save model and exit
#must save model with name "model.h5", so the video file can find this specific file and run it
model.save('model.h5')
#exit()



KeyboardInterrupt: 